In [1]:
import numpy as np

## LRP on a simple Markov-chain NN

In [2]:
# define widths of input, hidden and output layers
layer_widths = [2,2]
num_layers = len(layer_widths)

# one transition (weight) matrix is of shape (layer_width x layer_width). We need (num_layers - 1) of them
np.random.seed(5)
weights = []
for i in range(len(layer_widths) - 1):
    W = np.random.uniform(size=(layer_widths[i+1], layer_widths[i]))
    W /= W.sum(axis=0, keepdims=True)
    
    # manual overwrite:
    W = np.array([[.8, .3],
                  [.2, .7]])
    weights.append(W)
    
    print(f"Weight matrix from layer {i} to {i+1}:")
    print(W, '\n')

# every weight matrix is a stochastic transition matrix:

Weight matrix from layer 0 to 1:
[[0.8 0.3]
 [0.2 0.7]] 



In [3]:
# lets "classify" something
x = np.array([2., 0.])
x /= x.sum()
assert(len(x) == layer_widths[0])
activations = [x]

for W in weights:
    activations.append(W @ activations[-1])
    print(activations[-1].sum())
    
activations

# the activations sum to 1 in every layer:

1.0


[array([1., 0.]), array([0.8, 0.2])]

In [5]:
# lets 'explain' the input using LRP-0 rule. we offer the standard way of initializating output layer relevancies R, and others.

def LRP(weights, activations, initialization_method="default", verbose=False, return_coefficients=False):
    num_layers = len(activations)
    assert(len(weights) + 1 == len(activations))
    
    R = [None]*num_layers
    if (initialization_method == "default"):
        R[-1] = activations[-1] * (activations[-1] == activations[-1].max())
    elif (initialization_method == "j_star_eq_1"):
        R[-1] = 1. * (activations[-1] == activations[-1].max())
    elif (initialization_method == "output_activations"):
        R[-1] = activations[-1]
    else:
        raise NotImplementedError

    print(f"R initialized with method {initialization_method}:", R)
    
    # store matrices of LRP coefficients
    coefficients = []
    
    for l in range(num_layers-2, -1, -1):
        if (verbose):
            print("\ncalculating relevance for layer", l)
            print(f"relevance for layer {l+1} was \n", R[l+1])
            print("weights \n", weights[l])
            print(W.shape, h.shape, h[None, :].shape)

        W = weights[l]     # weights from the current to the deeper layer
        h = activations[l] # activations of the current layer
        R_prev = R[l+1]    # Relevance score of the deeper layer
        
        # C (vector form, one entry per entry in R[l+1])
        C = W * h[None, :]
        C /= C.sum(axis=1, keepdims=True)
        
        coefficients.insert(0, C)
        
        R[l] = C.T @ R_prev
        
    if return_coefficients==True:
        return coefficients
    
    return R

### Lets run LRP with different R_T initializations - how do the rel

In [10]:
R = LRP(weights, activations, initialization_method="output_activations")

print()
print("R:          ", R)
print("activations:", activations)

R initialized with method output_activations: [None, array([0.8, 0.2])]

R:           [array([1., 0.]), array([0.8, 0.2])]
activations: [array([1., 0.]), array([0.8, 0.2])]


In [11]:
R_default = LRP(weights, activations, initialization_method="default")

print()
print("R:   ", R_default)

R initialized with method default: [None, array([0.8, 0. ])]

R:    [array([0.8, 0. ]), array([0.8, 0. ])]


In [12]:
R_j_star_eq_1 = LRP(weights, activations, initialization_method="j_star_eq_1")

print()
print("R:          ", R_j_star_eq_1)

R initialized with method j_star_eq_1: [None, array([1., 0.])]

R:           [array([1., 0.]), array([1., 0.])]


In [13]:
print("R_j_star_eq_1:         ", R_j_star_eq_1)
print("R_default, normalized: ", [unscaled / unscaled.sum() for unscaled in R_default])

R_j_star_eq_1:          [array([1., 0.]), array([1., 0.])]
R_default, normalized:  [array([1., 0.]), array([1., 0.])]


## In a Markov Chain NN, are the LRP weightings, computed for one input, in general the right inverse weight matrix?

In [17]:

%load_ext autoreload
%autoreload 2

from helpers import h0_W_h1_C_R0
import matplotlib.pyplot as plt

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [18]:
# define widths of input, hidden and output layers
layer_widths = [2,2]
num_layers = len(layer_widths)

# one transition (weight) matrix is of shape (layer_width x layer_width). We need (num_layers - 1) of them
np.random.seed(56)
weights = []
for i in range(len(layer_widths) - 1):
    W = np.random.uniform(size=(layer_widths[i+1], layer_widths[i]))
    W[0,0] += 3
    W /= W.sum(axis=0, keepdims=True)
    weights.append(W)
    
    print(f"Weight matrix from layer {i} to {i+1}:")
    print(W, '\n')

# every weight matrix is a stochastic transition matrix:

Weight matrix from layer 0 to 1:
[[0.85536345 0.62931404]
 [0.14463655 0.37068596]] 



In [19]:
# lets "classify" something
x = np.array([2., 1.])
x /= x.sum()
assert(len(x) == layer_widths[0])
activations = [x]

for W in weights:
    activations.append(W @ activations[-1])
    print(activations[-1].sum())
    
activations

# the activations sum to 1 in every layer:

1.0


[array([0.66666667, 0.33333333]), array([0.78001365, 0.21998635])]

In [20]:
coefficients = LRP(weights, activations, return_coefficients=True) # note: init method doesn't matter, coefficients are the same.
coefficients

R initialized with method default: [None, array([0.78001365, 0.        ])]


[array([[0.73106708, 0.26893292],
        [0.43831976, 0.56168024]])]

In [21]:
inverse_weights = [np.linalg.inv(W) for W in weights]
inverse_weights

[array([[ 1.63984481, -2.78396667],
        [-0.63984481,  3.78396667]])]

In [22]:
forw0 = np.array([1, 5]).astype(float)
forw0 /= forw0.sum()
forw0 = x

forw1 = weights[0] @ forw0
# forw2 = weights[1] @ forw1

forw0

array([0.66666667, 0.33333333])

In [23]:
# lrp2 = forw2
lrp1 = forw1 # coefficients[1].T @ lrp2
lrp0 = coefficients[0].T @ lrp1
lrp0

array([0.66666667, 0.33333333])

In [24]:
# inv2 = forw2
inv1 = forw1 # inverse_weights[1] @ inv2
inv0 = inverse_weights[0] @ inv1
inv0

array([0.66666667, 0.33333333])